In [1]:
import matplotlib.pyplot as plt
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
import tensorflow as tf
import numpy as np
import glob
import os
import sys  
import cv2
from tqdm import tqdm
import random
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn import preprocessing
import tensorflow as tf
import scipy

from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


In [2]:
print(tf.__version__)
data_path="npz_data/violence/*"
# data_path2="npz_data/testtalk/*"

img_size=(224,224,3)
frame_number=20
num_class=2



1.13.1


In [3]:
def load_data(data_path):
    data=np.empty([1,frame_number,img_size[0],img_size[1],3])
    label=np.empty([1,num_class])
    file_list=glob.glob(data_path)
    data_list=[d for d in file_list if "data" in d.split("/")[-1]]
    label_list=[l for l in file_list if "label" in l.split("/")[-1]]
#     for index in range(len(data_list)):
#         d=np.load(data_list[index])["arr_0"]
#         l=np.load(label_list[index])["arr_0"]
#         data=np.append(data,d,axis=0)
#         data=np.append(label,l,axis=0)
#     data=data[1:]
#     label=label[1:]
    for index in range(len(data_list)):
        data=np.load(data_list[index])["arr_0"]
        label=np.load(label_list[index])["arr_0"]
    return data,label

data,label=load_data(data_path)

print(data.shape)

#try to upsize the data
sample=np.empty([len(data),frame_number,img_size[0],img_size[1],3])

print(sample.shape)

for frameCubeNum in range(len(data)):
    frameCube = data[frameCubeNum,...]
    frame_stack=np.empty([frame_number,img_size[0],img_size[1],3])
#     print("size of frame cube {}".format(frame_stack.shape))

    for imgcubeNum in range(len(frameCube)):
        imgcube = frameCube[imgcubeNum,...]
        img_stack=np.empty([img_size[0],img_size[1],3])
#         print("size of imgcube cube {}".format(img_stack.shape))

        for imglayerNum in range(3):
            imglayer = imgcube[...,imglayerNum]
            img_sm = cv2.resize(imglayer, (img_size[0],img_size[1]), interpolation=cv2.INTER_CUBIC)
#             print("size of imglayer {}".format(imglayer.shape))
            img_stack[...,imglayerNum] = img_sm
        
        frame_stack[imgcubeNum,...]=img_stack
#         frame_stack=np.append(frame_stack,img_stack,axis=0)
#         print(frameCube.shape)
        
    sample[frameCubeNum,...]=frame_stack
    
    
    
data = sample

print(data.shape)
print(label.shape)

(294, 20, 28, 28, 3)
(294, 20, 224, 224, 3)
(294, 20, 224, 224, 3)
(294, 2)


In [4]:
# #load extra test data
# def load_test_data(data_path):
#     data=np.empty([1,frame_number,img_size[0],img_size[1],3])
#     label=np.empty([1,num_class])
#     file_list=glob.glob(data_path)
#     data_list=[d for d in file_list if "data" in d.split("/")[-1]]
#     label_list=[l for l in file_list if "label" in l.split("/")[-1]]
# #     for index in range(len(data_list)):
# #         d=np.load(data_list[index])["arr_0"]
# #         l=np.load(label_list[index])["arr_0"]
# #         data=np.append(data,d,axis=0)
# #         data=np.append(label,l,axis=0)
# #     data=data[1:]
# #     label=label[1:]
#     for index in range(len(data_list)):
#         data=np.load(data_list[index])["arr_0"]
#         label=np.load(label_list[index])["arr_0"]
#     return data,label

# test2data,test2label=load_test_data(data_path2)

# print(test2data.shape)

# #try to upsize the data
# sample2=np.empty([len(test2data),frame_number,img_size[0],img_size[1],3])

# print(sample2.shape)

# for frameCubeNum in range(len(test2data)):
#     frameCube = test2data[frameCubeNum,...]
#     frame_stack=np.empty([frame_number,img_size[0],img_size[1],3])
# #     print("size of frame cube {}".format(frame_stack.shape))

#     for imgcubeNum in range(len(frameCube)):
#         imgcube = frameCube[imgcubeNum,...]
#         img_stack=np.empty([img_size[0],img_size[1],3])
# #         print("size of imgcube cube {}".format(img_stack.shape))

#         for imglayerNum in range(3):
#             imglayer = imgcube[...,imglayerNum]
#             img_sm = cv2.resize(imglayer, (img_size[0],img_size[1]), interpolation=cv2.INTER_CUBIC)
# #             print("size of imglayer {}".format(imglayer.shape))
#             img_stack[...,imglayerNum] = img_sm
        
#         frame_stack[imgcubeNum,...]=img_stack
# #         frame_stack=np.append(frame_stack,img_stack,axis=0)
# #         print(frameCube.shape)
        
#     sample2[frameCubeNum,...]=frame_stack
    
    
    
# test2data = sample2

# print(test2data.shape)
# print(test2label.shape)

In [5]:
# train_x = data[:int(label.shape[0]*0.8),...]
# train_y = label[:int(label.shape[0]*0.8),...]
# valid_x = data[int(label.shape[0]*0.8):int(label.shape[0]*0.9),...]
# valid_y = label[int(label.shape[0]*0.8):int(label.shape[0]*0.9),...]
# test_x = data[int(label.shape[0]*0.9):,...]
# test_y = label[int(label.shape[0]*0.9):,...]

train_x, valid_x, train_y,valid_y =  train_test_split(data, label, test_size=0.5,random_state=42)
test_x, valid_x, test_y,valid_y =  train_test_split(valid_x, valid_y, test_size=0.5, random_state=42)

del data,label

In [6]:
print(test_y.shape)


(73, 2)


In [7]:
import tensorflow_hub as hub


W0514 17:35:35.688138 140397181781760 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [8]:
# learning_rate = 1e-6
X = tf.placeholder(tf.float32,[None,frame_number,224,224,3])
labels = tf.placeholder(tf.float32,[None,num_class])


module = hub.Module("https://tfhub.dev/deepmind/i3d-kinetics-600/1",trainable=False)
outputs = module(X)

dense1 = tf.layers.dense(inputs=outputs, units=128)
dropped1 = tf.nn.dropout(dense1,keep_prob=0.5)
dense2 = tf.layers.dense(inputs=dropped1, units=128)
dropped2 = tf.nn.dropout(dense2,keep_prob=0.5)
# dense2 = tf.layers.dense(inputs=dense1, units=128)
# dense3 = tf.layers.dense(inputs=dense2, units=128)

logits = tf.layers.dense(inputs=dropped2, units=num_class)

prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=labels))
optimizer = tf.train.AdamOptimizer()
train_op = optimizer.minimize(loss_op)
        
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

saver = tf.train.Saver()

  

Instructions for updating:
Colocations handled automatically by placer.


W0514 17:35:37.594489 140397181781760 deprecation.py:323] From /home/jetherng/keras/lib/python3.5/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0514 17:35:38.936826 140397181781760 saver.py:1483] Saver not created because there are no variables in the graph to restore


Instructions for updating:
Use keras.layers.dense instead.


W0514 17:35:39.012495 140397181781760 deprecation.py:323] From <ipython-input-8-a8abe9461434>:9: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


W0514 17:35:39.024795 140397181781760 deprecation.py:506] From <ipython-input-8-a8abe9461434>:10: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



W0514 17:35:39.055184 140397181781760 deprecation.py:323] From <ipython-input-8-a8abe9461434>:22: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [9]:
batch_size = 2
num_steps = int(train_x.shape[0]/batch_size)
display_step = 2 #was 2 when achieved test2 accuracy 0.667

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for epoch in range(10):   #was 10
        print("epoch {}".format(epoch))

        for step in range(0, num_steps):
            batch_x, batch_y = train_x[step:(step+1)*batch_size,...],train_y[step:(step+1)*batch_size,...]

                # Run optimization op (backprop)
            sess.run(train_op, feed_dict={X: batch_x, labels: batch_y})
            if step % display_step == 0 or step == 1:
                    # Calculate batch loss and accuracy
                loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                         labels: batch_y})
                print("Step " + str(step) + ", Minibatch Loss= " + \
                          "{:.4f}".format(loss) + ", Training Accuracy= " + \
                          "{:.3f}".format(acc))


        print("Optimization Finished fpr epoch {}\n".format(epoch))
        valaccuracy =sess.run(accuracy, feed_dict={X: valid_x,
                                              labels:valid_y});
        print("validation Accuracy: {}".format(valaccuracy))
            
        f = open("result.txt","a+")
        f.write("validation accuracy for epoch {} is {}\n".format(epoch,valaccuracy))
        f.close()
    save_path = saver.save(sess, "/tmp/model.ckpt")

    del train_x,train_y,valid_x,valid_y
    testaccuracy = sess.run(accuracy, feed_dict={X: test_x,
                                              labels:test_y})
    print("Testing accuracy is {} \n".format(testaccuracy))
    f = open("result.txt","a+")
    f.write("testing accuracy is {}\n".format(testaccuracy))
    f.close()
    
    del test_x,test_y
#     print(test2label)
#     testaccuracy2 = sess.run(accuracy, feed_dict={X: test2data,
#                                               labels:test2label})
#     print("Testing2 accuracy is {} \n".format(testaccuracy2))
#     f = open("result.txt","a+")
#     f.write("testing2 accuracy is {}\n".format(testaccuracy2))
#     f.close()

 







epoch 0
Step 0, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 1, Minibatch Loss= 89.4407, Training Accuracy= 0.333
Step 2, Minibatch Loss= 196.6114, Training Accuracy= 0.750
Step 4, Minibatch Loss= 146.8102, Training Accuracy= 0.667
Step 6, Minibatch Loss= 218.0002, Training Accuracy= 0.625
Step 8, Minibatch Loss= 337.5642, Training Accuracy= 0.500
Step 10, Minibatch Loss= 216.3310, Training Accuracy= 0.500
Step 12, Minibatch Loss= 620.8678, Training Accuracy= 0.357
Step 14, Minibatch Loss= 227.2975, Training Accuracy= 0.562
Step 16, Minibatch Loss= 330.6172, Training Accuracy= 0.500
Step 18, Minibatch Loss= 322.5207, Training Accuracy= 0.600
Step 20, Minibatch Loss= 315.7881, Training Accuracy= 0.455
Step 22, Minibatch Loss= 88.8907, Training Accuracy= 0.708
Step 24, Minibatch Loss= 198.0201, Training Accuracy= 0.692
Step 26, Minibatch Loss= 125.2279, Training Accuracy= 0.643
Step 28, Minibatch Loss= 320.8257, Training Accuracy= 0.367
Step 30, Minibatch Loss= 181.1575, Trainin

ResourceExhaustedError: OOM when allocating tensor with shape[74,64,10,112,112] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node module_apply_default/RGB/inception_i3d/Conv3d_1a_7x7/conv_3d/convolution (defined at /home/jetherng/keras/lib/python3.5/site-packages/tensorflow_hub/native_module.py:547) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'module_apply_default/RGB/inception_i3d/Conv3d_1a_7x7/conv_3d/convolution', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/jetherng/keras/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/jetherng/keras/lib/python3.5/site-packages/traitlets/config/appl

In [ ]:
# tf.reset_default_graph()

# saver = tf.train.Saver()

# with tf.Session() as sess:
#     saver = tf.train.import_meta_graph('/tmp/model.ckpt.meta')
#     saver.restore(sess, "/tmp/model.ckpt")    
#     print("Model restored.")

